In [ ]:
# 控制算法对比



In [1]:
import numpy as np
import os

from rm_control.simulation.sim_interface import SimInterface
from rm_control.controllers.controllers import *
from rm_control.planning.trajectory_generator import SineWaveTrajectory
from rm_control.utils.plotter import BenchmarkPlotter
from rm_control.assets import get_model_path_xml, get_model_path_urdf
from rm_control.dynamics.pinocchio_dynamics import PinocchioDynamics
from rm_control.results import get_result_fig_dir

from rm_control.utils.benchmark import BenchmarkRunner

xml_path = get_model_path_xml()
result_fig_dir = get_result_fig_dir()
sim = SimInterface(xml_path, dt=0.001, render=False)

urdf_path = get_model_path_urdf()
pin_dyn = PinocchioDynamics(urdf_path, ee_name="panda_link7")


✅ 锁定末端执行器: link7 (ID: 8)
🤖 SimInterface 初始化完成. 活跃自由度: 7
ℹ️ [Pinocchio] 未指定关节列表，默认加载 Panda 前 7 轴模式。
✅ 模型裁剪完成! 原自由度: 9 -> 现自由度: 7


In [8]:
trajectory = SineWaveTrajectory(freq=1.0, amp=0.1, duration=5.0, dt=0.001)

kp_val = 100.0
kd_val = 20.0

# 全关节统一！CTC 的数学美感就在于此
kp = np.array([kp_val] * 7)
kd = np.array([kd_val] * 7)

# 把 sim 和 controller 传进去
# ctrl_with_grav = JointPDController(kp, kd, pin_dyn=None)
sim.reset()
ctc_ctrl = ComputedTorqueController(kp, kd, pin_dyn=pin_dyn)
runner = BenchmarkRunner(sim, ctc_ctrl)

# 开始跑
history_data = runner.run(trajectory, duration=5.0, visualize=False)

# =========================================================
# 3. 后处理画图
# =========================================================
result_fig_name = 'crontrol_test_'+ctc_ctrl.name+".png"
result_fig_path = os.path.join(result_fig_dir, result_fig_name)
plotter = BenchmarkPlotter(run_name="final_test2")
plotter.plot(history_data, save_path=result_fig_path)



🔄 SimInterface: Environment Reset.
🚀 开始测试: Computed_Torque_Control (可视化: False)
🛠️  模式切换至: TORQUE
⚡ 正在进行极速仿真 (Headless mode)...
⏱️ 仿真完成！实际耗时: 0.4807s (仿真时长: 5.0s)
✅ 测试任务完成。

📊 --- final_test2 跟踪误差统计 (单位: 度) ---
关节       | 平均误差(RMSE)      | 最大误差(Max)      
--------------------------------------------------
Joint 1  | 0.5193          | 1.0978         
Joint 2  | 0.1845          | 0.6249         
Joint 3  | 0.3412          | 0.6280         
Joint 4  | 0.2984          | 0.4823         
Joint 5  | 2.9511          | 5.3658         
Joint 6  | 2.8176          | 4.6851         
Joint 7  | 3.8162          | 5.6200         
--------------------------------------------------
综合平均误差 (Total RMSE): 1.5612 度

💾 正在保存全关节结果图表到: /Users/chenxu/Library/CloudStorage/OneDrive-Personal/Code/robot_dynamic_ctl_demo/rm_control/results/figures/crontrol_test_Computed_Torque_Control.png ...
✅ 保存成功！


In [7]:
trajectory = SineWaveTrajectory(freq=1.0, amp=0.1, duration=5.0, dt=0.001)

# 笛卡尔参数: 前3个是位置(x,y,z)，后3个是旋转(rx,ry,rz)
# 单位: N/m (位置), Nm/rad (旋转)
kp_cart = np.array([800.0, 800.0, 800.0, 150.0, 150.0, 150.0])
kd_cart = np.array([ 50.0,  50.0,  50.0, 10.0, 10.0, 10.0])

# 实例化控制器
imp_ctrl = CartesianImpedanceController(kp_cart, kd_cart, pin_dyn)
sim.reset()
runner = BenchmarkRunner(sim, imp_ctrl)

# 开始跑
history_data = runner.run(trajectory, duration=5.0, visualize=False)

# 后处理画图
result_fig_name = 'control_test_' + imp_ctrl.name + ".png"
result_fig_path = os.path.join(result_fig_dir, result_fig_name)
plotter = BenchmarkPlotter(run_name="impedance_test")
plotter.plot(history_data, save_path=result_fig_path)

🔄 SimInterface: Environment Reset.
🚀 开始测试: Cartesian_Impedance (可视化: False)
🛠️  模式切换至: TORQUE
⚡ 正在进行极速仿真 (Headless mode)...
⏱️ 仿真完成！实际耗时: 0.1776s (仿真时长: 5.0s)
✅ 测试任务完成。

📊 --- impedance_test 跟踪误差统计 (单位: 度) ---
关节       | 平均误差(RMSE)      | 最大误差(Max)      
--------------------------------------------------
Joint 1  | 2.7532          | 4.8051         
Joint 2  | 2.5704          | 4.0933         
Joint 3  | 3.2614          | 6.0201         
Joint 4  | 2.9813          | 4.4472         
Joint 5  | 2.3301          | 4.3032         
Joint 6  | 1.3873          | 2.1728         
Joint 7  | 3.6303          | 6.4478         
--------------------------------------------------
综合平均误差 (Total RMSE): 2.7020 度

💾 正在保存全关节结果图表到: /Users/chenxu/Library/CloudStorage/OneDrive-Personal/Code/robot_dynamic_ctl_demo/rm_control/results/figures/control_test_Cartesian_Impedance.png ...
✅ 保存成功！


In [ ]:
# 2. 设置轨迹
trajectory = SineWaveTrajectory(freq=1.0, amp=0.1, duration=5.0, dt=0.001)

# OSC 参数: 代表期望加速度增益 (Kp = omega^2)
# 设带宽为 10 rad/s -> Kp=100, Kd=20
kp_osc = np.array([100.0] * 6)
kd_osc = np.array([ 20.0] * 6)


# 实例化控制器
osc_ctrl = OperationalSpaceController(kp_osc, kd_osc, pin_dyn)
sim.reset()
runner = BenchmarkRunner(sim, osc_ctrl)

# 开始跑
history_data = runner.run(trajectory, duration=5.0, visualize=False)

# 后处理画图
result_fig_name = 'control_test_' + osc_ctrl.name + ".png"
result_fig_path = os.path.join(result_fig_dir, result_fig_name)
plotter = BenchmarkPlotter(run_name="osc_test")
plotter.plot(history_data, save_path=result_fig_path)

🔄 SimInterface: Environment Reset.
🚀 开始测试: Operational_Space_Control (可视化: False)
🛠️  模式切换至: TORQUE
⚡ 正在进行极速仿真 (Headless mode)...
⏱️ 仿真完成！实际耗时: 0.4854s (仿真时长: 5.0s)
✅ 测试任务完成。

📊 --- osc_test 跟踪误差统计 (单位: 度) ---
关节       | 平均误差(RMSE)      | 最大误差(Max)      
--------------------------------------------------
Joint 1  | 2.5827          | 3.6304         
Joint 2  | 1.9703          | 3.4424         
Joint 3  | 4.6922          | 8.2068         
Joint 4  | 3.6564          | 5.5538         
Joint 5  | 4.8203          | 8.8738         
Joint 6  | 4.6578          | 7.0723         
Joint 7  | 4.5116          | 7.4842         
--------------------------------------------------
综合平均误差 (Total RMSE): 3.8416 度

💾 正在保存全关节结果图表到: /Users/chenxu/Library/CloudStorage/OneDrive-Personal/Code/robot_dynamic_ctl_demo/rm_control/results/figures/control_test_Operational_Space_Control.png ...
✅ 保存成功！
